In [1]:
%cd /home/akashchaurasia/Utkarsh/Learning_Projects/LIME

/home/akashchaurasia/Utkarsh/Learning_Projects/LIME


https://towardsdatascience.com/interpreting-image-classification-model-with-lime-1e7064a2f2e5

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,DirectoryIterator
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import backend as K
import os
import tensorflow as tf

# Path to train and test directory
dir_ = os.path.join('dog-cat-panda/')


# Generate training and test data with Image Generator
train_datagen = ImageDataGenerator(rescale=1/255,
                                  validation_split = 0.2)


train_generator = train_datagen.flow_from_directory(dir_,target_size=(100, 100),
                                                   batch_size= 1920,
                                                   class_mode='categorical',
                                                   shuffle=False,
                                                   subset = 'training')

test_generator = train_datagen.flow_from_directory(dir_,
                                                          target_size = (100,100),
                                                          batch_size = 480,
                                                          class_mode = 'categorical',
                                                          shuffle=False,
                                                          subset = 'validation')


# Fetch the data and the labels
x_train, y_train = next(train_generator)
x_test, y_test  = next(test_generator)

# Fix the filepath
test_filepath = []
for filepath in test_generator.filepaths:
    filepath = filepath.replace('\\', '/')
    test_filepath.append(filepath)

2022-12-28 17:40:34.306886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 17:40:36.327881: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-28 17:40:36.327905: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-28 17:40:38.336526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [2]:
# model = Sequential([
    
#     # First convolution
#         Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
#         MaxPooling2D(2, 2),
    
#     # Second convolution
#         Conv2D(32, (3,3), activation='relu'),
#         MaxPooling2D(2,2),
    
#     # Third convolution
#         Conv2D(64, (3,3), activation='relu'),
#         MaxPooling2D(2,2),
    
#         Flatten(),
    
#     # Dense hidden layer
#         Dense(512, activation='relu'),
#         Dropout(0.2),
    
#     # Output neuron. 
#         Dense(3, activation='softmax') 
# ])
# model.summary()

In [6]:
model = Sequential([
    
    # First convolution
        Conv2D(256, (3,3), activation='relu', input_shape=(100, 100, 3)),
        MaxPooling2D(2, 2),
    
    # Second convolution
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        BatchNormalization(),
        Dropout(0.5),
    
    # Third convolution
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        BatchNormalization(),
        Dropout(0.5),

        Conv2D(256, (3,3), activation='relu'),
        MaxPooling2D(2,2),
    
        Flatten(),
    
    # Dense hidden layer
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
    
    # Output neuron. 
        Dense(3, activation='softmax') 
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 256)       7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 23, 23, 128)      512       
 ormalization)                                                   
                                                        

2022-12-28 17:16:40.949333: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-28 17:16:40.949414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (utkarshchalsey): /proc/driver/nvidia/version does not exist
2022-12-28 17:16:40.952435: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model = Sequential([
    Conv2D(256, (3,3), padding='same', activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2,2), strides=2),
    BatchNormalization(),

    Conv2D(128, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2), strides=2),
    BatchNormalization(),
 

    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2), strides=2),
    BatchNormalization(),


    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2), strides=2),
    BatchNormalization(),

    
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2), strides=2),
    BatchNormalization(),

    # Conv2D(128, (2,2), padding='same', activation='relu'),
    # MaxPooling2D((2,2), strides=2),
    # BatchNormalization(),
    
    # Conv2D(256, (2,2), padding='same', activation='relu'),
    # MaxPooling2D((2,2), strides=2),
    # BatchNormalization(),
    
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.8),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 100, 100, 256)     7168      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 50, 50, 256)      0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 50, 50, 256)      1024      
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 50, 50, 128)       295040    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 25, 25, 128)      0         
 2D)                                                             
                                                      

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator,epochs=50,verbose=1) 

Epoch 1/50


2022-12-28 17:53:23.458728: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 230400000 exceeds 10% of free system memory.
2022-12-28 17:53:28.155455: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 57600000 exceeds 10% of free system memory.
2022-12-28 17:53:28.182109: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 4915200000 exceeds 10% of free system memory.
